In [1]:
import numpy as np
import pandas as pd
import numpy as np
import sys

In [2]:
d = {'OrderNo' : [], 'PartNo' : [], 'BendingLines' : [], 'WeldingPoints' : [], 'PaintTime' : [], 'DueDate' : []}
orderDF = pd.DataFrame(data=d)
jobs = []
path = r"C:\Users\vd5t\Desktop\QCP\OrdersGenerated.txt"
orders = open(path, "r")
orders = orders.readlines()
orders = orders[1:]
for i in range(len(orders)):
    orders[i] = orders[i].split(', ')
    actualOrder = orders[i]
    newjob = tuple((int(actualOrder[0]),int(actualOrder[1]),int(actualOrder[2]),int(actualOrder[3]),int(0),int(actualOrder[5])))
    jobs.append(newjob)
    newRow = {'OrderNo' : actualOrder[0], 
              'PartNo' : actualOrder[1], 
              'BendingLines' : actualOrder[2], 
              'WeldingPoints' : actualOrder[3], 
              'PaintTime' : actualOrder[4], 
              'DueDate' : actualOrder[5]}
    orderDF = orderDF.append(newRow, ignore_index = True)    

orderDF = orderDF.astype('int64')

#for simplicity the following lines:
orderDF = orderDF.iloc[0:3][:]

print (jobs)

[(0, 0, 1, 5, 0, 303), (0, 1, 8, 7, 0, 303), (1, 0, 6, 14, 0, 345), (1, 1, 7, 1, 0, 345), (2, 0, 4, 4, 0, 123), (3, 0, 6, 13, 0, 324), (4, 0, 3, 2, 0, 47), (4, 1, 5, 11, 0, 47), (5, 0, 5, 0, 0, 365), (6, 0, 4, 8, 0, 172), (6, 1, 4, 3, 0, 172), (6, 2, 2, 0, 0, 172), (7, 0, 4, 0, 0, 347), (8, 0, 4, 5, 0, 344), (8, 1, 4, 13, 0, 344), (8, 2, 3, 10, 0, 344), (9, 0, 1, 4, 0, 241), (10, 0, 2, 10, 0, 289), (10, 1, 5, 11, 0, 289), (10, 2, 2, 4, 0, 289), (11, 0, 4, 11, 0, 142), (12, 0, 4, 12, 0, 95), (12, 1, 1, 6, 0, 95), (12, 2, 8, 4, 0, 95), (13, 0, 5, 8, 0, 98), (13, 1, 5, 2, 0, 98), (14, 0, 4, 0, 0, 83), (14, 1, 6, 12, 0, 83)]


In [3]:
#job = "OrderNo, PartNo, BendingLines, WeldingPoints, PaintTime, DueDate
jobs = [(0, 0, 2, 0, 0, 1), (0, 1, 1, 1, 0, 2)]  
# zwei aufeinanderfolgende Operation (Biegen und Schweißen) am selben Bauteil zum Test von H1
# ein anderes Bauteil das zur selben Zeit auf der selben Maschine laufen kann wie bei dem anderen zum Test H2
# Strafe für Zeitüberschr.



In [4]:
bend = [0, 1, 2]
solder = [3, 4]
paint =  [5]

bend_dauer = 2
weld_dauer = 3
paint_dauer = 6
t_step = 1

In [5]:
def max_time(jobs):
    a = 0 
    b = 0 
    c = 0
    for i in jobs:
        actualTime = i[2]
        if a < b and a < c:
            a += actualTime
        elif b < a and b < c:
            b += actualTime
        else:
            c += actualTime
    maxB = max(a, b, c) * 2
    
    a = 0
    b = 0
    for i in jobs:
        actualTime = i[3]
        if a < b:
            a += actualTime
        else:
            b += actualTime
    maxW = max(a, b) * 3
    
    maximumPainting = 0
    for i in jobs:
        maximumPainting += i[4]
    maxP = maximumPainting * 6
    
    return maxB + maxW + maxP



In [6]:
# OrderNo, PartNo, BendingLines, WeldingPoints, PaintTime, DueDate 
m_time = max_time(jobs) * 10

m_t_steps = int(m_time / 10)
    
print('The maximal maketime for the given operations is: ' + str(m_time) + ' second(s).\nWhich is equal to: ' + str(m_t_steps) + ' time steps.\n\n')


The maximal maketime for the given operations is: 90 second(s).
Which is equal to: 9 time steps.




In [7]:
def ops(jobs, bend, solder, paint, m_time):
    ops = []
    for x in jobs:
        if x[2] > 0:
            punish = x[2] * 2
            if x[3] > 0:
                punish += x[3] * 3
            if x[4] > 0:
                punish += x[4] * 6
            for m in bend:
                t = 0
                while t <= m_t_steps - punish:
                    ops.append(tuple((x[0], x[1], x[2], m, t)))
                    t += t_step
                    
        if x[3] > 0:
            punish = x[3] * 3
            if x[4] > 0:
                punish += x[4] * 6
            for m in solder:
                t = 0
                if x[2] > 0:
                    t += x[2] * 2
                while t <= m_t_steps - punish:
                    ops.append(tuple((x[0], x[1], x[3], m, t)))
                    t += t_step

        if x[4] > 0:
            punish = x[4] * 6
            for m in paint:
                t = 0
                if x[2] > 0:
                    t += x[2] * 2
                if x[3] > 0:
                    t += x[3] * 3
                while t <= m_t_steps - punish:
                    ops.append(tuple((x[0], x[1], x[4], m, t)))
                    t += t_step
        # ops.append(('||||||||||||END OF JOB ' + str(x[0]) + ' OPERATION ' + str(x[1]) + '||||||||||||'))
    return ops

In [8]:
ops = ops(jobs, bend, solder, paint, m_time)

print('Anzahl an Kombinationen: ' + str(len(ops)) + '\n')

print('Possible combiniations:\n')
print('Format: OrderNo, PartNo, Op, Machine, Time')

print(ops)

Anzahl an Kombinationen: 43

Possible combiniations:

Format: OrderNo, PartNo, Op, Machine, Time
[(0, 0, 2, 0, 0), (0, 0, 2, 0, 1), (0, 0, 2, 0, 2), (0, 0, 2, 0, 3), (0, 0, 2, 0, 4), (0, 0, 2, 0, 5), (0, 0, 2, 1, 0), (0, 0, 2, 1, 1), (0, 0, 2, 1, 2), (0, 0, 2, 1, 3), (0, 0, 2, 1, 4), (0, 0, 2, 1, 5), (0, 0, 2, 2, 0), (0, 0, 2, 2, 1), (0, 0, 2, 2, 2), (0, 0, 2, 2, 3), (0, 0, 2, 2, 4), (0, 0, 2, 2, 5), (0, 1, 1, 0, 0), (0, 1, 1, 0, 1), (0, 1, 1, 0, 2), (0, 1, 1, 0, 3), (0, 1, 1, 0, 4), (0, 1, 1, 1, 0), (0, 1, 1, 1, 1), (0, 1, 1, 1, 2), (0, 1, 1, 1, 3), (0, 1, 1, 1, 4), (0, 1, 1, 2, 0), (0, 1, 1, 2, 1), (0, 1, 1, 2, 2), (0, 1, 1, 2, 3), (0, 1, 1, 2, 4), (0, 1, 1, 3, 2), (0, 1, 1, 3, 3), (0, 1, 1, 3, 4), (0, 1, 1, 3, 5), (0, 1, 1, 3, 6), (0, 1, 1, 4, 2), (0, 1, 1, 4, 3), (0, 1, 1, 4, 4), (0, 1, 1, 4, 5), (0, 1, 1, 4, 6)]


In [9]:
 def convert(ord1, part1, op1, mach1, t1):
    return ops.index(tuple((ord1, part1, op1, mach1, t1)))

In [10]:
# a = convert(0, 1, 1, 2, 2)
# print(a)

In [11]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

In [12]:
QUBO = np.zeros((len(ops), len(ops)))
np.set_printoptions(suppress=True)

QUBO_pr = str(QUBO).replace('\n', ' ').replace(']', ']\n').replace(']\n]', ']]').replace('  ', ' ').replace(' -', '-').replace('[[', '[\n[').replace(' [', '[').replace(']]', ']\n]')
print(QUBO_pr)

[
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  

In [13]:
#Format: OrderNo, PartNo, Op, Machine, Time 
    
def fill_QUBO_with_indexes(QUBO, ord1, part1, op1, mach1, t1, ord2, part2, op2, mach2, t2, value):
    index_a = convert(ord1, part1, op1, mach1, t1)
    index_b = convert(ord2, part2, op2, mach2, t2)
    if index_a > index_b:
        index_a, index_b = index_b, index_a
    QUBO[index_a][index_b] += value

Constraint 1 (each operation 'o' of each job 'j' exactly once on one machine 'm' at a certaint (starting-)time 't'):

$$
H_{0}:=\sum_{o \in 0}\left(\sum_{m \in M_{o}} \sum_{t=0}^{T-1} x_{j, o, m, t}-1\right)^{2}
$$


In [14]:
alpha = 1

In [15]:
# H0: Mehrmaliges Abarbeiten der selben Operation eines Bauteils eines Jobs bestrafen
'ops: OrderNo, PartNo, OpNum, Machine, Time'

for x in jobs:
    if x[2] > 0:
        punish = x[2] * 2
        if x[3] > 0:
            punish += x[3] * 3
        if x[4] > 0:
            punish += x[4] * 6
        for m in bend:
            t = 0
            while t <= m_t_steps - punish:
                u = t
                while u <= m_t_steps - punish:
                    if t != u:
                        fill_QUBO_with_indexes(QUBO, x[0], x[1], x[2], m, t, x[0], x[1], x[2], m, u, -alpha)
                    else:
                        fill_QUBO_with_indexes(QUBO, x[0], x[1], x[2], m, t, x[0], x[1], x[2], m, u, 2*alpha)
                    u += t_step
                t += t_step

    if x[3] > 0:
        punish = x[3] * 3
        if x[4] > 0:
            punish += x[4] * 6
        for m in solder:
            t = 0
            if x[2] > 0:
                t += x[2] * 2
            while t <= m_t_steps - punish:
                u = t
                while u <= m_t_steps - punish:
                    if t != u:
                        fill_QUBO_with_indexes(QUBO, x[0], x[1], x[3], m, t, x[0], x[1], x[3], m, u, -alpha)
                    else:
                        fill_QUBO_with_indexes(QUBO, x[0], x[1], x[3], m, t, x[0], x[1], x[3], m, u, 2*alpha)
                    u += t_step
                t += t_step
                        
    if x[4] > 0:
        punish = x[4] * 6
        for m in paint:
            t = 0
            if x[2] > 0:
                t += x[2] * 2
            if x[3] > 0:
                t += x[3] * 3
            while t <= m_t_steps - punish:
                u = t
                while u <= m_t_steps - punish:
                    if t != u:
                        fill_QUBO_with_indexes(QUBO, x[0], x[1], x[4], m, t, x[0], x[1], x[4], m, u, -alpha)
                    else:
                        fill_QUBO_with_indexes(QUBO, x[0], x[1], x[4], m, t, x[0], x[1], x[4], m, u, 2*alpha)
                    u += t_step
                t += t_step


In [16]:
QUBO_pr = str(QUBO).replace('\n', ' ').replace(']', ']\n').replace(']\n]', ']]').replace('  ', ' ').replace(' -', '-').replace(' [ ', '[').replace('[ ', '[').replace('[[', '[\n[').replace(']]', ']\n]').replace(' [', '[')
print(QUBO_pr)

[
[2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 2.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 2.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 2.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0.

In [17]:
#Ok.

Secondly, for every job $j \in J$, its operations $O_j$ must be executed in the order given by 

<br /><br />
$$
\left.c\right|_{o_{j}}: O_{j} \rightarrow\left\{1, \ldots, n_{j}\right\}
$$
<br />
$$H_{1}:=\sum_{j \in J}\left(\sum_{k_{n-1}<i<k_{n}, t+p_{i} > t'} x_{i, t}x_{i+1, t'}\right)$$


In [18]:
# H1 verbotene Reihenfolgen innerhalb eines Bauteils eines Jobs bestrafen
'ops: OrderNo, PartNo, OpNum, Machine, Time'

beta = 1

for tupel1 in ops:
    if tupel1[3] <= 3:
        dom = 2
    elif tupel1[3] <= 5:
        dom = 3
    else:
        dom = 0
    for tupel2 in ops:
        if (tupel1[0] == tupel2[0]) and (tupel1[1] == tupel2[1]):
            if (tupel2[3] > 2 and 2 >= tupel1[3] and tupel2[4] < (tupel1[4] + dom)) or (tupel2[3] > 4 and 4 >= tupel1[3] and tupel1[3] > 2 and tupel2[4] < (tupel1[4] + dom)):
                fill_QUBO_with_indexes(QUBO, *tupel1, *tupel2, beta)

In [19]:
QUBO_pr = str(QUBO).replace('\n', ' ').replace(']', ']\n').replace(']\n]', ']]').replace('  ', ' ').replace(' -', '-').replace(' [ ', '[').replace('[ ', '[').replace('[[', '[\n[').replace(']]', ']\n]').replace(' [', '[')
print(QUBO_pr)

[
[2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 2.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 2.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 2.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0.

In [20]:
# H2 Geichzeitiges Laufen zweier verschiedener Operationen auf der selben Ma
'ops: OrderNo, PartNo, OpNum, Machine, Time'

gamma = 1

for tupel1 in ops:
    if tupel1[3] <= 3:
        dom = 2
    elif tupel1[3] <= 5:
        dom = 3
    else:
        dom = 0
    for tupel2 in ops:
        if ((tupel1[0] != tupel2[0] or tupel1[1] != tupel2[1]) and tupel1[3] == tupel2[3] and tupel1[4] <= tupel2[4] < tupel1[4] + dom):
            fill_QUBO_with_indexes(QUBO, *tupel1, *tupel2, gamma)

In [21]:
QUBO_pr = str(QUBO).replace('\n', ' ').replace(']', ']\n').replace(']\n]', ']]').replace('  ', ' ').replace(' -', '-').replace(' [ ', '[').replace('[ ', '[').replace('[[', '[\n[').replace(']]', ']\n]').replace(' [', '[')
print(QUBO_pr)

[
[2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 2.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 2.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 2.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0.

In [22]:
# K Bestrafung für Verspätung
'ops: OrderNo, PartNo, OpNum, Machine, Time'
'jobs: OrderNo, PartNo, BendingLines, WeldingPoints, PaintTime, DueDate'

delta = 1
     
c = 0
while c < len(jobs):
    t1 = 0
    while t1 < len(ops):
        if jobs[c][0] == ops[t1][0]:
            tail = 0
            punish = 0
            if ops[t1][3] < 3: # op is bend
                dom = 2 * ops[t1][2]
                if jobs[c][3] > 0: # there is still welding to do
                    tail += jobs[c][3] * 3
                if jobs[c][4] > 0: # has paint op
                    punish += jobs[c][4] * 6
            else: # op is weld
                dom = 3 * ops[t1][2]
                if jobs[c][4] > 0: # has paint op
                    punish += jobs[c][4] * 6
            if ops[t1][4] + dom + tail > jobs[c][5]*6 - punish:
                fill_QUBO_with_indexes(QUBO, *ops[t1], *ops[t1], delta)
        t1 += 1
    c+=1

In [23]:
QUBO_pr = str(QUBO).replace('\n', ' ').replace(']', ']\n').replace(']\n]', ']]').replace('  ', ' ').replace(' -', '-').replace(' [ ', '[').replace('[ ', '[').replace('[[', '[\n[').replace(']]', ']\n]').replace(' [', '[')
print(QUBO_pr)

[
[2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 2.-1.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 3.-1.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 1. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 3.-1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 1. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 2. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 2.-1.-1.-1.-1. 0. 0. 0. 0. 0. 0.  0. 0. 0. 0.

In [24]:
qubo_d = {}
x1 = 0
while x1 < len(ops):
    x2 = x1
    while x2 < len(ops):
        qubo_d[(x1, x2)] = QUBO[x1][x2]
        x2 += 1
    x1 += 1
    
print(qubo_d)

{(0, 0): 2.0, (0, 1): -1.0, (0, 2): -1.0, (0, 3): -1.0, (0, 4): -1.0, (0, 5): -1.0, (0, 6): 0.0, (0, 7): 0.0, (0, 8): 0.0, (0, 9): 0.0, (0, 10): 0.0, (0, 11): 0.0, (0, 12): 0.0, (0, 13): 0.0, (0, 14): 0.0, (0, 15): 0.0, (0, 16): 0.0, (0, 17): 0.0, (0, 18): 2.0, (0, 19): 1.0, (0, 20): 0.0, (0, 21): 0.0, (0, 22): 0.0, (0, 23): 0.0, (0, 24): 0.0, (0, 25): 0.0, (0, 26): 0.0, (0, 27): 0.0, (0, 28): 0.0, (0, 29): 0.0, (0, 30): 0.0, (0, 31): 0.0, (0, 32): 0.0, (0, 33): 0.0, (0, 34): 0.0, (0, 35): 0.0, (0, 36): 0.0, (0, 37): 0.0, (0, 38): 0.0, (0, 39): 0.0, (0, 40): 0.0, (0, 41): 0.0, (0, 42): 0.0, (1, 1): 2.0, (1, 2): -1.0, (1, 3): -1.0, (1, 4): -1.0, (1, 5): -1.0, (1, 6): 0.0, (1, 7): 0.0, (1, 8): 0.0, (1, 9): 0.0, (1, 10): 0.0, (1, 11): 0.0, (1, 12): 0.0, (1, 13): 0.0, (1, 14): 0.0, (1, 15): 0.0, (1, 16): 0.0, (1, 17): 0.0, (1, 18): 1.0, (1, 19): 2.0, (1, 20): 1.0, (1, 21): 0.0, (1, 22): 0.0, (1, 23): 0.0, (1, 24): 0.0, (1, 25): 0.0, (1, 26): 0.0, (1, 27): 0.0, (1, 28): 0.0, (1, 29): 0.0, (